<a href="https://colab.research.google.com/github/darchon30704/primers-sars-cov-2/blob/main/Summary_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PATH = '/content/drive/MyDrive/Colab Notebooks/Genomic/primers-sars-cov-2/data_1503_featureSpace'

# features.py

In [ ]:
import copy
import datetime
import graphviz
import logging
import numpy as np
import os
import sys
import sklearn

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier

from sklearn.multiclass import OneVsOneClassifier 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import RadiusNeighborsClassifier

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

# used for normalization
from sklearn.preprocessing import StandardScaler

# used for cross-validation
from sklearn.model_selection import StratifiedKFold

# this is an incredibly useful function
from pandas import read_csv
import pandas as pd 
def loadDatasetOriginal(run) :
	
	# data used for the predictions
	dfData = read_csv( PATH +"/data/data_0.csv", header=None, sep=',')
	dfLabels = read_csv( PATH +"/data/labels.csv", header=None)
	biomarkers = read_csv( PATH +"/data/features_0.csv", header=None)

	# create folder
	folderName =PATH + "/run"+str(run)+"/"
	if not os.path.exists(folderName) : os.makedirs(folderName)

	pd.DataFrame(dfData.values).to_csv( PATH+ "/run"+str(run)+"/data_0.csv", header=None, index =None)
	pd.DataFrame(biomarkers.values.ravel()).to_csv( PATH+ "/run"+str(run)+"/features_0.csv", header=None, index =None)
	pd.DataFrame(dfLabels.values.ravel()).to_csv( PATH+ "/run"+str(run)+"/labels.csv", header=None, index =None)
	return dfData.values, dfLabels.values.ravel(), biomarkers.values.ravel() # to have it in the format that the classifiers like


def loadDataset(globalIndex, run) :
	
	# data used for the predictions
	dfData = read_csv( PATH + "/run"+str(run)+"/data_"+str(globalIndex)+".csv", header=None, sep=',')
	dfLabels = read_csv( PATH + "/run"+str(run)+"/labels.csv", header=None)
	biomarkers = read_csv( PATH + "/run"+str(run)+"/features_"+str(globalIndex)+".csv", header=None)

	return dfData.values, dfLabels.values.ravel(), biomarkers.values.ravel() # to have it in the format that the classifiers like

# this function returns a list of features, in relative order of importance
def relativeFeatureImportance(classifier) :
	
	# this is the output; it will be a sorted list of tuples (importance, index)
	# the index is going to be used to find the "true name" of the feature
	orderedFeatures = []

	# the simplest case: the classifier already has a method that returns relative importance of features
	if hasattr(classifier, "feature_importances_") :

		orderedFeatures = zip(classifier.feature_importances_ , range(0, len(classifier.feature_importances_)))
		orderedFeatures = sorted(orderedFeatures, key = lambda x : x[0], reverse=True)
	
	# some classifiers are ensembles, and if each element in the ensemble is able to return a list of feature importances
	# (that are going to be all scored following the same logic, so they could be easily aggregated, in theory)
	elif hasattr(classifier, "estimators_") and hasattr(classifier.estimators_[0], "feature_importances_") :

		# add up the scores given by each estimator to all features
		global_score = np.zeros(classifier.estimators_[0].feature_importances_.shape[0])

		for estimator in classifier.estimators_ :
			for i in range(0, estimator.feature_importances_.shape[0]) :
				global_score[i] += estimator.feature_importances_[i]

		# "normalize", dividing by the number of estimators
		for i in range(0, global_score.shape[0]) : global_score[i] /= len(classifier.estimators_)

		# proceed as above to obtain the ranked list of features
		orderedFeatures = zip(global_score, range(0, len(global_score)))
		orderedFeatures = sorted(orderedFeatures, key = lambda x : x[0], reverse=True)
	
	# the classifier does not have "feature_importances_" but can return a list
	# of all features used by a lot of estimators (typical of ensembles)
	elif hasattr(classifier, "estimators_features_") :

		numberOfFeaturesUsed = 0
		featureFrequency = dict()
		for listOfFeatures in classifier.estimators_features_ :
			for feature in listOfFeatures :
				if feature in featureFrequency :
					featureFrequency[feature] += 1
				else :
					featureFrequency[feature] = 1
			numberOfFeaturesUsed += len(listOfFeatures)
		
		for feature in featureFrequency : 
			featureFrequency[feature] /= numberOfFeaturesUsed

		# prepare a list of tuples (name, value), to be sorted
		orderedFeatures = [ (featureFrequency[feature], feature) for feature in featureFrequency ]
		orderedFeatures = sorted(orderedFeatures, key=lambda x : x[0], reverse=True)

	# the classifier does not even have the "estimators_features_", but it's
	# some sort of linear/hyperplane classifier, so it does have a list of
	# coefficients; for the coefficients, the absolute value might be relevant
	elif hasattr(classifier, "coef_") :
	
		# now, "coef_" is usually multi-dimensional, so we iterate on
		# all dimensions, and take a look at the features whose coefficients
		# more often appear close to the top; but it could be mono-dimensional,
		# so we need two special cases
		dimensions = len(classifier.coef_.shape)
		#print("dimensions=", len(dimensions))
		featureFrequency = None # to be initialized later
		
		# check on the dimensions
		if dimensions == 1 :
			featureFrequency = np.zeros(len(classifier.coef_))
			
			relativeFeatures = zip(classifier.coef_, range(0, len(classifier.coef_)))
			relativeFeatures = sorted(relativeFeatures, key=lambda x : abs(x[0]), reverse=True)
			
			for index, values in enumerate(relativeFeatures) :
				value, feature = values
				featureFrequency[feature] += 1/(1+index)

		elif dimensions > 1 :
			featureFrequency = np.zeros(len(classifier.coef_[0]))
			
			# so, for each dimension (corresponding to a class, I guess)
			for i in range(0, len(classifier.coef_)) :
				# we give a bonus to the feature proportional to
				# its relative order, good ol' 1/(1+index)
				relativeFeatures = zip(classifier.coef_[i], range(0, len(classifier.coef_[i])))
				relativeFeatures = sorted(relativeFeatures, key=lambda x : abs(x[0]), reverse=True)
				
				for index, values in enumerate(relativeFeatures) :
					value, feature = values
					featureFrequency[feature] += 1/(1+index)
			
		# finally, let's sort
		orderedFeatures = [ (featureFrequency[feature], feature) for feature in range(0, len(featureFrequency)) ]
		orderedFeatures = sorted(orderedFeatures, key=lambda x : x[0], reverse=True)

	else :
		print("The classifier does not have any way to return a list with the relative importance of the features")

	return np.array(orderedFeatures)

def featureSelection(globalIndex, variableSize,run) :
	
	# a few hard-coded values
	numberOfFolds = 10
	
	
	# list of classifiers, selected on the basis of our previous paper "
	classifierList = [
			# ensemble
			#[AdaBoostClassifier(), "AdaBoostClassifier"],
			#[AdaBoostClassifier(n_estimators=300), "AdaBoostClassifier(n_estimators=300)"],
			#[AdaBoostClassifier(n_estimators=1500), "AdaBoostClassifier(n_estimators=1500)"],
			#[BaggingClassifier(), "BaggingClassifier"],
			
			[GradientBoostingClassifier(n_estimators=300), "GradientBoostingClassifier(n_estimators=300)"],
			[RandomForestClassifier(n_estimators=300), "RandomForestClassifier(n_estimators=300)"],
			[LogisticRegression(), "LogisticRegression"],
			[PassiveAggressiveClassifier(),"PassiveAggressiveClassifier"],
			[SGDClassifier(), "SGDClassifier"],
			[SVC(kernel='linear'), "SVC(linear)"],
			[RidgeClassifier(), "RidgeClassifier"],
			[BaggingClassifier(n_estimators=300), "BaggingClassifier(n_estimators=300)"],
			#[ExtraTreesClassifier(), "ExtraTreesClassifier"],
			#[ExtraTreesClassifier(n_estimators=300), "ExtraTreesClassifier(n_estimators=300)"],
			#[GradientBoostingClassifier(), "GradientBoostingClassifier"], # features_importances_
			#[GradientBoostingClassifier(n_estimators=300), "GradientBoostingClassifier(n_estimators=300)"],
			#[GradientBoostingClassifier(n_estimators=1000), "GradientBoostingClassifier(n_estimators=1000)"],
			#[RandomForestClassifier(), "RandomForestClassifier"],
			#[RandomForestClassifier(n_estimators=300), "RandomForestClassifier(n_estimators=300)"],
			#[RandomForestClassifier(n_estimators=1000), "RandomForestClassifier(n_estimators=1000)"], # features_importances_

			# linear
			#[ElasticNet(), "ElasticNet"],
			#[ElasticNetCV(), "ElasticNetCV"],
			#[Lasso(), "Lasso"],
			#[LassoCV(), "LassoCV"],
			#[LogisticRegression(), "LogisticRegression"], # coef_
			#[LogisticRegressionCV(), "LogisticRegressionCV"],
			#[PassiveAggressiveClassifier(), "PassiveAggressiveClassifier"], # coef_
			#[RidgeClassifier(), "RidgeClassifier"], # coef_
			#[RidgeClassifierCV(), "RidgeClassifierCV"],
			#[SGDClassifier(), "SGDClassifier"], # coef_
			#[SVC(kernel='linear'), "SVC(linear)"], # coef_, but only if the kernel is linear...the default is 'rbf', which is NOT linear
			
			# naive Bayes
			#[BernoulliNB(), "BernoulliNB"],
			#[GaussianNB(), "GaussianNB"],
			#[MultinomialNB(), "MultinomialNB"],
			
			# neighbors
			#[KNeighborsClassifier(), "KNeighborsClassifier"], # no way to return feature importance
			# TODO this one creates issues
			#[NearestCentroid(), "NearestCentroid"], # it does not have some necessary methods, apparently
			#[RadiusNeighborsClassifier(), "RadiusNeighborsClassifier"],
			
			# tree
			#[DecisionTreeClassifier(), "DecisionTreeClassifier"],
			#[ExtraTreeClassifier(), "ExtraTreeClassifier"],

			]
	
	# this is just a hack to check a few things
	#classifierList = [
	#	   [RandomForestClassifier(), "RandomForestClassifier"]
	#	   ]

	print("Loading dataset...")
	if (globalIndex==0):
		X, y, biomarkerNames = loadDatasetOriginal(run)
	else:
		X, y, biomarkerNames = loadDataset(globalIndex, run)
	
	
	
	numberOfTopFeatures=int(variableSize)
	# create folder
	folderName = PATH + "/run"+str(run)+"/"
	if not os.path.exists(folderName) : os.makedirs(folderName)
	
	# prepare folds
	skf = StratifiedKFold(n_splits=numberOfFolds, shuffle=True)
	indexes = [ (training, test) for training, test in skf.split(X, y) ]
	
	# this will be used for the top features
	topFeatures = dict()
	
	# iterate over all classifiers
	classifierIndex = 0
	
	globalAccuracy=0;
	
	for originalClassifier, classifierName in classifierList :
		
		print("\nClassifier " + classifierName)
		classifierPerformance = []
		classifierTopFeatures = dict()

		# iterate over all folds
		for train_index, test_index in indexes :
			
			X_train, X_test = X[train_index], X[test_index]
			y_train, y_test = y[train_index], y[test_index]
			
			# let's normalize, anyway
			scaler = StandardScaler()
			X_train = scaler.fit_transform(X_train)
			X_test = scaler.transform(X_test)

			classifier = copy.deepcopy(originalClassifier)
			classifier.fit(X_train, y_train)
			scoreTraining = classifier.score(X_train, y_train)
			scoreTest = classifier.score(X_test, y_test)
			
			print("\ttraining: %.4f, test: %.4f" % (scoreTraining, scoreTest))
			classifierPerformance.append( scoreTest )
			
			# now, let's get a list of the most important features, then mark the ones in the top X
			orderedFeatures = relativeFeatureImportance(classifier) 
			for i in range(0, numberOfTopFeatures) :
				
				feature = int(orderedFeatures[i][1])

				if feature in topFeatures :
					topFeatures[ feature ] += 1
				else :
					topFeatures[ feature ] = 1
				
				if feature in classifierTopFeatures :
					classifierTopFeatures[ feature ] += 1
				else :
					classifierTopFeatures[ feature ] = 1
			
		line ="%s\t%.4f\t%.4f\n" % (classifierName, np.mean(classifierPerformance), np.std(classifierPerformance))
		
		globalAccuracy=globalAccuracy+np.mean(classifierPerformance)
		
		print(line)
		fileName=folderName+"results.txt"
		fo = open(fileName, 'a')
		fo.write( line )
		fo.close()
		# save most important features for the classifier
		with open( os.path.join(folderName, classifierName + ".csv"), "w" ) as fp :
	
			fp.write("feature,frequencyInTop" + str(numberOfTopFeatures) + "\n")
			
			# transform dictionary into list
			listOfClassifierTopFeatures = [ (key, classifierTopFeatures[key]) for key in classifierTopFeatures ]
			listOfClassifierTopFeatures = sorted( listOfClassifierTopFeatures, key = lambda x : x[1], reverse=True )
			
			for feature, frequency in listOfClassifierTopFeatures :
				fp.write( str(biomarkerNames[feature]) + "," + str(float(frequency/numberOfFolds)) + "\n")
	
	# save most important features overall
	with open( os.path.join(folderName, "global_"+str(int(globalIndex))+".csv"), "w" ) as fp :
		
		fp.write("feature,frequencyInTop" + str(numberOfTopFeatures) + "\n")
		
		# transform dictionary into list
		listOfTopFeatures = [ (key, topFeatures[key]) for key in topFeatures ]
		listOfTopFeatures = sorted( listOfTopFeatures, key = lambda x : x[1], reverse=True )
		
		tempIndex=0
		for feature, frequency in listOfTopFeatures :
			if tempIndex<numberOfTopFeatures:
				fp.write( str(biomarkerNames[feature]) + "," + str(float(frequency/numberOfFolds)) + "\n")
			tempIndex=tempIndex+1
	globalAccuracy=globalAccuracy/8
	return globalAccuracy

# reduceData.py

In [ ]:
def reduceDataset(globalIndex, run) :
	
	# data used for the predictions
	if(globalIndex==0):
		dfData = read_csv(PATH + "/data/data_"+str(globalIndex)+".csv", header=None, sep=',')
		ids=read_csv(PATH + "/data/features_"+str(globalIndex)+".csv", header=None, sep=',')
	else:
		dfData = read_csv( PATH + "/run"+str(run)+"/data_"+str(globalIndex)+".csv", header=None, sep=',')
		ids=read_csv( PATH + "/run"+str(run)+"/features_"+str(globalIndex)+".csv", header=None, sep=',')
	
	idsRed=read_csv( PATH + "/run"+str(run)+"/global_"+str(globalIndex)+".csv", sep=',')
	
	data=dfData.values
	idsRed=idsRed.values
	
	ids=ids.values

	print("data Y %d" %(len(data)))
	print("data X %d" %(len(data[0])))
	print(len(ids))
	print(len(idsRed))
	
	tempIds=[]
	for i in range(0,len(idsRed)):
		if (idsRed[i,1]>=1):
			tempIds.append(idsRed[i,0])
	print(len(tempIds))
	count=0
	
	dataRed=np.zeros((len(data),len(tempIds)))

	for i in range(0,len(tempIds)):
		for j in range (0,len(ids)):
			if (ids[j]== tempIds[i]):
				count=count+1
				for k in range(0,len(data)):
					dataRed[k,i]=data[k,j]
	
	
	pd.DataFrame(tempIds).to_csv(PATH + "/run"+str(run)+"/features_"+str(globalIndex+1)+".csv", header=None, index =None)
	pd.DataFrame(dataRed).to_csv(PATH + "/run"+str(run)+"/data_"+str(globalIndex+1)+".csv", header=None, index =None)			
	print(count)
	
	return len(ids),len(tempIds)


# classifiersMulti.py

In [ ]:
import copy
import datetime
import graphviz
import logging
import numpy as np
import os
import sys
import pandas as pd 

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier

from sklearn.multiclass import OneVsOneClassifier 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import RadiusNeighborsClassifier

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

# used for normalization
from sklearn.preprocessing import  Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# used for cross-validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from scipy import interp
import matplotlib.pyplot as plt
# this is an incredibly useful function
from pandas import read_csv

from statsmodels.multivariate.manova import MANOVA
import statsmodels.api as sm
from scipy import stats
def loadDataset2() :
	
	# data used for the predictions
	dfData = read_csv(PATH + "/src/best/data_0.csv", header=None, sep=',')
	dfLabels = read_csv(PATH + "/src/best/labels.csv", header=None)
		
	return dfData.values.ravel(), dfLabels.values.ravel() # to have it in the format that the classifiers like


def runFeatureReduce() :
	
	# a few hard-coded values
	numberOfFolds = 10
	
	# list of classifiers, selected on the basis of our previous paper "
	classifierList = [
		
			[GradientBoostingClassifier(n_estimators=300), "GradientBoostingClassifier(n_estimators=300)"],
			[RandomForestClassifier(n_estimators=300), "RandomForestClassifier(n_estimators=300)"],
			[LogisticRegression(solver='lbfgs',), "LogisticRegression"],
			[PassiveAggressiveClassifier(),"PassiveAggressiveClassifier"],
			[SGDClassifier(), "SGDClassifier"],
			[SVC(kernel='linear'), "SVC(linear)"],
			[RidgeClassifier(), "RidgeClassifier"],
			[BaggingClassifier(n_estimators=300), "BaggingClassifier(n_estimators=300)"],
			# ensemble
			#[AdaBoostClassifier(), "AdaBoostClassifier"],
			#[AdaBoostClassifier(n_estimators=300), "AdaBoostClassifier(n_estimators=300)"],
			#[AdaBoostClassifier(n_estimators=1500), "AdaBoostClassifier(n_estimators=1500)"],
			#[BaggingClassifier(), "BaggingClassifier"],
			
			#[ExtraTreesClassifier(), "ExtraTreesClassifier"],
			#[ExtraTreesClassifier(n_estimators=300), "ExtraTreesClassifier(n_estimators=300)"],
			 # features_importances_
			#[GradientBoostingClassifier(n_estimators=300), "GradientBoostingClassifier(n_estimators=300)"],
			#[GradientBoostingClassifier(n_estimators=1000), "GradientBoostingClassifier(n_estimators=1000)"],
			
			#[RandomForestClassifier(n_estimators=300), "RandomForestClassifier(n_estimators=300)"],
			#[RandomForestClassifier(n_estimators=1000), "RandomForestClassifier(n_estimators=1000)"], # features_importances_

			# linear
			#[ElasticNet(), "ElasticNet"],
			#[ElasticNetCV(), "ElasticNetCV"],
			#[Lasso(), "Lasso"],
			#[LassoCV(), "LassoCV"],
			 # coef_
			#[LogisticRegressionCV(), "LogisticRegressionCV"],
			  # coef_
			 # coef_
			#[RidgeClassifierCV(), "RidgeClassifierCV"],
			 # coef_
			 # coef_, but only if the kernel is linear...the default is 'rbf', which is NOT linear
			
			# naive Bayes
			#[BernoulliNB(), "BernoulliNB"],
			#[GaussianNB(), "GaussianNB"],
			#[MultinomialNB(), "MultinomialNB"],
			
			# neighbors
			#[KNeighborsClassifier(), "KNeighborsClassifier"], # no way to return feature importance
			# TODO this one creates issues
			#[NearestCentroid(), "NearestCentroid"], # it does not have some necessary methods, apparently
			#[RadiusNeighborsClassifier(), "RadiusNeighborsClassifier"],
			
			# tree
			#[DecisionTreeClassifier(), "DecisionTreeClassifier"],
			#[ExtraTreeClassifier(), "ExtraTreeClassifier"],

			]
	
	# this is just a hack to check a few things
	#classifierList = [
	#		[RandomForestClassifier(), "RandomForestClassifier"]
	#		]

	print("Loading dataset...")
	X, y = loadDataset2()
	
	print(len(X))
	print(len(X[0]))
	print(len(y))

	
	labels=np.max(y)+1
	# prepare folds
	skf = StratifiedKFold(n_splits=numberOfFolds, shuffle=True)
	indexes = [ (training, test) for training, test in skf.split(X, y) ]
	
	# this will be used for the top features
	topFeatures = dict()
	
	# iterate over all classifiers
	classifierIndex = 0
	
	
	
	
	for originalClassifier, classifierName in classifierList :
		
		print("\nClassifier " + classifierName)
		classifierPerformance = []

		cMatrix=np.zeros((labels, labels))
		# iterate over all folds
		
		indexFold = 0

		yTest=[]
		yNew=[]

		for train_index, test_index in indexes :
			
			X_train, X_test = X[train_index], X[test_index]
			y_train, y_test = y[train_index], y[test_index]
			
			# let's normalize, anyway
			# MinMaxScaler StandardScaler Normalizer
			scaler = StandardScaler()
			X_train = scaler.fit_transform(X_train)
			X_test = scaler.transform(X_test)

		
			
			classifier = copy.deepcopy(originalClassifier)
			classifier.fit(X_train, y_train)
			scoreTraining = classifier.score(X_train, y_train)
			scoreTest = classifier.score(X_test, y_test)
			
			y_new = classifier.predict(X_test)
			
			
			
			yNew.append(y_new)
			yTest.append(y_test)
			
			for i in range(0,len(y_new)):
				cMatrix[y_test[i]][y_new[i]]+=1


			print("\ttraining: %.4f, test: %.4f" % (scoreTraining, scoreTest))
			classifierPerformance.append( scoreTest )

		pd.DataFrame(cMatrix).to_csv(PATH + "/src/best/cMatrix"+str(classifierIndex)+".csv", header=None, index =None)
		classifierIndex+=1
		line ="%s \t %.4f \t %.4f \n" % (classifierName, np.mean(classifierPerformance), np.std(classifierPerformance))
		
		
		print(line)
		fo = open(PATH + "/src/best/results.txt", 'a')
		fo.write( line )
		fo.close()
	
	return


# aBioInf100.py 

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
#from runClassifiers import *
import time
import tqdm
from tqdm import tqdm

In [ ]:
def mainRun(indexRun) :
	run=indexRun
	start_time = time.time()
	globalAnt=0.0
	globalIndex=0
	globalAccuracy=0.0

	X, y, biomarkerNames = loadDatasetOriginal(run)
	
	if (int(len(y))>1000):
		numberOfTopFeatures = 1000
	else :
		numberOfTopFeatures = int(len(X[0])*0.80)

	variableSize=numberOfTopFeatures;
	while True:
		globalAnt=globalAccuracy
		globalAccuracy=featureSelection(globalIndex,variableSize, run)
		print(globalAccuracy)
		print(globalIndex)
		print(variableSize)
		size,sizereduced=reduceDataset(globalIndex, run)
		
		if((globalAccuracy<(0.50)) and (globalIndex!=0)):
			break
		if(variableSize==0):
			break
		variableSize=int(variableSize*0.80)
		
		globalIndex=globalIndex+1
	elapsed_time = time.time() - start_time
	print("time")
	print(elapsed_time)
	return

def main():
	threads=5
	totalRuns=10
	Parallel(n_jobs=threads, verbose=5, backend="multiprocessing")(delayed(mainRun)(i) for i in tqdm(range(0,totalRuns)))
	return


In [ ]:
main()

  0%|          | 0/10 [00:00<?, ?it/s][Parallel(n_jobs=5)]: Using backend MultiprocessingBackend with 5 concurrent workers.


Loading dataset...
Loading dataset...
Loading dataset...
Loading dataset...
Loading dataset...

Classifier GradientBoostingClassifier(n_estimators=300)

Classifier GradientBoostingClassifier(n_estimators=300)

Classifier GradientBoostingClassifier(n_estimators=300)
Classifier GradientBoostingClassifier(n_estimators=300)


Classifier GradientBoostingClassifier(n_estimators=300)
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Streaming output truncated to the last 5000 lines.

	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000

Classifier RidgeClassifier
	training: 0.9981, test: 1.0000
	training: 1.0000, test: 0.9828
	training: 0.9981, test: 1.0000
	training: 1.0000, test: 1.0000
PassiveAggressiveClassifier	0.9983	0.0052
	training: 1.0000, test: 0.9831

	training: 0.9981, test: 1.0000
	training: 0.9962, test: 0.9828
	training: 0.9981, test: 1.0000

Classifier SGDClassifier
	training: 0.9981, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 0.9981, test: 1.0000
	training: 0.9981, test: 1.0000
	training: 0.9962, test: 0.9828
RidgeClassifier	0.9949	0.0079
	training: 1.0000, test: 1.0000

	training: 1.0000, test: 1.0000

Classifier BaggingClassifier(n_estimators=300)
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 1.0000
	training: 1.0000, test: 0.9828
	training: 1.0000, test: 1.0000
SGDC

[Parallel(n_jobs=5)]: Done   7 out of  10 | elapsed: 279.1min remaining: 119.6min


	training: 0.8514, test: 0.8448
	training: 0.8514, test: 0.8448
BaggingClassifier(n_estimators=300)	0.8508	0.0102

0.846938924605494
31
1
data Y 583
data X 2
2
1
1
1
Loading dataset...

Classifier GradientBoostingClassifier(n_estimators=300)
	training: 0.8495, test: 0.8621
	training: 0.8531, test: 0.8305
	training: 0.8514, test: 0.8448
	training: 0.6870, test: 0.6780
	training: 0.8514, test: 0.8448
BaggingClassifier(n_estimators=300)	0.8508	0.0102

0.846938924605494
31
1
data Y 583
data X 2
2
1
1
1
Loading dataset...

Classifier GradientBoostingClassifier(n_estimators=300)
	training: 0.8511, test: 0.8475
	training: 0.6870, test: 0.6780
	training: 0.8495, test: 0.8621
	training: 0.6870, test: 0.6780
	training: 0.6870, test: 0.6780
	training: 0.8495, test: 0.8621
	training: 0.6870, test: 0.6780
	training: 0.6870, test: 0.6780
	training: 0.6857, test: 0.6897
	training: 0.8495, test: 0.8621
	training: 0.6857, test: 0.6897
	training: 0.6857, test: 0.6897
	training: 0.8514, test: 0.8448
	tra

[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed: 280.0min remaining:    0.0s
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed: 280.0min finished


# summaryMulti.py

In [ ]:
# used for normalization
from sklearn.preprocessing import  Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# used for cross-validation
from sklearn.model_selection import StratifiedKFold

# this is an incredibly useful function
from pandas import read_csv
import matplotlib.pyplot as plt

# directory="data"

def fakeBootStrapper():
	# create folder
	folderName =PATH + "/src/summary/best"
	if not os.path.exists(folderName) : os.makedirs(folderName)

	orig_stdout = sys.stdout
	f = open(PATH + '/src/best/out.txt', 'w')
	sys.stdout = f
	
	runs=5
	directory="run"+str(0)
	f=open(PATH +"/"+directory+"/results.txt", "r")
	fl =f.readlines()
	blocks=int(len(fl)/8)
	print(blocks)
	results=np.zeros((blocks,runs+1))
	for j in range(0,runs):
		directory="run"+str(j)
		f=open(PATH + "/"+directory+"/results.txt", "r")
		fl =f.readlines()
		blocks=int(len(fl)/8)
		count=0
		value=0
		variables=np.zeros(blocks)
		accuracy=np.zeros(blocks)
		indexResults=0
		for x in fl:
			a=x.split("\t")
			value=value+float(a[1])/8.0
			count=count+1
			if (count==8):
				accuracy[indexResults]=value
				value=0
				count=0
				indexResults=indexResults+1
		indexResults=0
		
		for i in range (0,blocks):
			dfFeats = (read_csv(PATH + "/"+directory+"/features_"+str(i)+".csv", header=None)).values.ravel() #.values.ravel() #.as_matrix().ravel() # to_numpy().ravel() #
			variables[i]=len(dfFeats)
			results[i,j+1]=accuracy[i]
			results[i,0]=variables[i]
		
		
	
	pd.DataFrame(results).to_csv(folderName+"/sum.csv", header=None, index =None)
	
	bestVal=np.zeros(runs)
	bestSize=np.zeros(runs)
	bestPos=np.zeros(runs)
	for j in range(0,runs):
		bestVal[j]=np.max(results[:,j+1])
	
	for j in range(0,runs):	
		for i in range (0,blocks):
			if ( bestVal[j]==results[i,j+1]):
				bestSize[j]=int(results[i,0])
				bestPos[j]=i
	print(bestVal)
	print(bestSize)
	print(bestPos)
	
	bestFeatures=[]
	signatures=[]
	for j in range(0,runs):
		dfFeats = read_csv(PATH + "/run"+str(j)+"/features_"+str(int(bestPos[j]))+".csv", header=None)
		bestFeatures.extend(dfFeats.values.ravel())
		signatures.append(dfFeats.values.ravel())
	#print(bestFeatures)
	signatures.append(bestVal)
	signatures.append(bestSize)
	pd.DataFrame(signatures).to_csv(folderName +"/signatures.csv", header=None, index =None)
	
	unique, counts = np.unique(bestFeatures, return_counts=True)
	
	resultsFeatures=np.zeros((len(unique),2), 'U16')
	for j in range(0,len(unique)):
		resultsFeatures[j,0]=unique[j]
		resultsFeatures[j,1]=counts[j]
	
	
	pd.DataFrame(resultsFeatures).to_csv(folderName+"/resultsFeatures.csv", header=None, index =None)
	
	print(np.max(bestVal))
	print(np.argmax(bestVal))
	print(int(bestPos[np.argmax(bestVal)]))
	
	runBest=int(np.argmax(bestVal))
	indexBest=int(bestPos[np.argmax(bestVal)])
	
	
	
	# data used for the predictions
	dfData = read_csv(PATH + "/run"+str(runBest)+"/data_"+str(indexBest)+".csv", header=None, sep=',')
	dfLabels = read_csv(PATH + "/run"+str(runBest)+"/labels.csv", header=None)
	biomarkers = read_csv(PATH + "/run"+str(runBest)+"/features_"+str(indexBest)+".csv", header=None)
	
	pd.DataFrame(dfData.values).to_csv(folderName + "/data_0.csv", header=None, index =None)
	pd.DataFrame(biomarkers.values.ravel()).to_csv(folderName + "/features_0.csv", header=None, index =None)
	pd.DataFrame(dfLabels.values.ravel()).to_csv(folderName + "/labels.csv", header=None, index =None)
	
	runFeatureReduce()
	sys.stdout = orig_stdout
	f.close()
	return

In [ ]:
(read_csv(PATH + "/run0/features_0.csv", header=None,  sep=','))

,0
0,AGGTAACAAACCAACCAACTT
1,CACGAGTAACTCGTCTATCTT
2,CACGTCCAACTCAGTTTGCCT
3,TTGCCTCAACTTGAACAGCCC
4,CGATCTAAAGTCATTTGACTT
...,...
3822,CAGCTATAATGCAGGTGTTCT
3823,GACGCATGGTTTAGCCAGCGT
3824,AAACAATGCTCTTCACTATGC
3825,CATTAATAATGCTGTTTACAC


In [ ]:
a,b = loadDataset2()

In [ ]:
fakeBootStrapper()

EmptyDataError: ignored